# AI-Assisted Literature Screening: A Comparative Analysis of GPT and DeepSeek



Daniela Agostina Gonzalez

> *Departamento de Economía y Finanzas, Facultad de Ciencias Económicas, Universidad Nacional de Córdoba, Córdoba, Argentina.*





## Preparing APIs

In [ ]:
pip install openai

In [ ]:
import pandas as pd
import numpy as np
import openai
import os
import json
import os
import pickle

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from openai import OpenAI
# Initialize the OpenAI client
client = OpenAI(
    api_key="<your api key>",
    #base_url="https://api.deepseek.com/beta", # When using DeepSeek
)

def get_completion(prompt, model="gpt-4o-mini", temperature=0): #change to  model="deepseek-chat", temperature=1 when using DeepSeek
    # Creating the completion using the new client object
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
    )
    # Check if completion is successful before accessing attributes
    if completion is not None and completion.choices and completion.choices[0].message:
        return completion.choices[0].message.content
    else:
        print(f"API call failed or returned unexpected data: {completion}, setting response to 'NA'")
        return "NA" # Return "NA" to continue the loop

## Abstracts Data

Sep, M. S. C., Vellinga, M., Sarabdjitsingh, R. A., & Joëls, M. (2021). The rodent object-in-context task: A systematic review and meta-analysis of important variables. PLOS ONE, 16(7). https://doi.org/10.1371/journal.pone.0253556.
Available at:
`https://github.com/asreview/synergy-dataset`


In [ ]:
# Randomize rows and save

# df = pd.read_csv(".../df_dataset_Sep_2021.csv")
# df = df.sample(frac=1)
# df.to_csv('.../shuffle_df_Sep_2021.csv')
df = pd.read_csv(".../shuffle_df_Sep_2021.csv")
df.head()

,Unnamed: 0,record_id,doi,title,abstract,label_included
0,157,128,https://doi.org/10.1016/s0896-6273(02)00684-0,Reversible Inhibition of CREB/ATF Transcriptio...,CREB is critical for long-lasting synaptic and...,0
1,22,12,https://doi.org/10.1002/hipo.22095,Lateral entorhinal cortex is critical for nove...,Episodic memory incorporates information about...,1
2,5,115,https://doi.org/10.1016/j.freeradbiomed.2016.1...,Cisplatin-induced mitochondrial dysfunction is...,Chemotherapy-related cognitive impairment (CRC...,1
3,58,231,https://doi.org/10.1038/sj.npp.1301401,Arousal and Stress Effects on Consolidation an...,This study examined the effects of the arousal...,0
4,198,120,https://doi.org/10.3389/fnbeh.2012.00011,The endocannabinoid transport inhibitor AM404 ...,Cannabinoid compounds may influence both emoti...,0


In [ ]:
abstract = df['abstract'].tolist()


In [ ]:
len(abstracts)

236

In [ ]:
# Creating a list for saving the results
list_key_model = list()

In [ ]:
for i in range(len(abstracts)):
    prompt = f"""
    Your task is to select papers that are relevant (1) or irrelevant (0), based on the abstracts, to make a systematic review on the field of behavioral neuroscience and cognitive psychology, focusing on context-dependent memory in rodents, particularly studied through the object-in-context task. The article will systematically review variations in the implementation of this task and analyzes factors influencing its outcomes, such as sex, testbox size, and trial delays. The paper aims to improve the design and reproducibility of future animal experiments related to memory by providing recommendations for standardizing procedures and variables in this context-based memory task.
    Give your answer in numbers, 1 if its relevant and 0 if its not. Do not include any other information, only the correspondent number
    For example:
    <abstract>: ```{abstracts[0]}```
    <response>: 0
    <abstract>: ```{abstracts[1]}```
    <response>: 1

    <abstract>: ```{abstracts[i]}```


    """

    response = get_completion(prompt)

    list_key_model.append(response)

    #print(lista_key[-1])

In [ ]:
# Append to the df and save
df['model_labels'] = [int(lista) for lista in list_key_model] # for each model
# df.to_csv('.../deep_gpt_shuffle_df_dataset_Sep_2021.csv')